# Temporary reading of pilot llava results

In [90]:
import pandas as pd

df = pd.read_csv("llava-100-test.csv")
df

,Unnamed: 0,question,response,reference,argument,substitution,exact_match/score
0,0,Is the curtain on the right side or on the lef...,right,right,curtain,NaN,1.0
1,1,Is the blind on the right side or on the left ...,right,right,curtain,blind,1.0
2,2,Is the protective covering on the right side o...,left,right,curtain,protective covering,0.0
3,3,Is the covering on the right side or on the le...,right,right,curtain,covering,1.0
4,4,Is the cloth on the right side or on the left ...,right,right,curtain,cloth,1.0
...,...,...,...,...,...,...,...
95,95,Are there vertebrates in the photo that are no...,no,no,zebra,vertebrate,1.0
96,96,Are there animals in the photo that are not st...,no,no,zebra,animal,1.0
97,97,Is the boy's hair curly and short?,yes,no,boy,NaN,0.0
98,98,Is the man's hair curly and short?,yes,no,boy,man,0.0


# Batch inference with Gemma/PaliGemma with HF + GCP

Gemma is a family of lightweight, state-of-the-art open models built from the same research and technology used to create the Gemini models, developed by Google DeepMind and other teams across Google. Text Generation Inference (TGI) is a toolkit developed by Hugging Face for deploying and serving LLMs, with high performance text generation. And, Google Vertex AI is a Machine Learning (ML) platform that lets you train and deploy ML models and AI applications, and customize large language models (LLMs) for use in your AI-powered applications. This example showcases how to deploy any supported text-generation model, in this case [`google/gemma-7b-it`](https://huggingface.co/google/gemma-7b-it), from the Hugging Face Hub on Vertex AI using the TGI DLC available in Google Cloud Platform (GCP).

![`google/gemma-7b-it` in the Hugging Face Hub](./assets/model-in-hf-hub.png)

## Setup / Configuration

First, you need to install `gcloud` in your local machine, which is the command-line tool for Google Cloud, following the instructions at [Cloud SDK Documentation - Install the gcloud CLI](https://cloud.google.com/sdk/docs/install).

Then, you also need to install the `google-cloud-aiplatform` Python SDK, required to programmatically create the Vertex AI model, register it, acreate the endpoint, and deploy it on Vertex AI.

In [1]:
!pip install --upgrade google-cloud-aiplatform


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


Optionally, to ease the usage of the commands within this tutorial, you need to set the following environment variables for GCP:

In [2]:
%env PROJECT_ID=multimodal-representations
%env LOCATION=us-central1
#%env CONTAINER_URI=us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-2.ubuntu2204.py310:latest
#%env CONTAINER_URI=us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-llava-serve
%env CONTAINER_URI=us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu124.2-3.ubuntu2204.py311



env: PROJECT_ID=multimodal-representations
env: LOCATION=us-central1
env: CONTAINER_URI=us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu124.2-3.ubuntu2204.py311


Then you need to login into your GCP account and set the project ID to the one you want to use to register and deploy the models on Vertex AI.

In [3]:
!gcloud auth login
!gcloud config set project $PROJECT_ID

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=UI5PYdVu9zvqCQZuLdHsOugJBfCmSp&access_type=offline&code_challenge=pSxNv7rhVp3FaVp1G6BYDsJx4oJ3y-w8KRT9HIfLCOw&code_challenge_method=S256


You are now logged in as [daliumuwork@gmail.com].
Your current project is [multimodal-representations].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].


Once you are logged in, you need to enable the necessary service APIs in GCP, such as the Vertex AI API, the Compute Engine API, and Google Container Registry related APIs.

**Warning:** Make sure, manually, that these are disabled after running exps (even though we will explicitly write code to disable them)

In [4]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable compute.googleapis.com
!gcloud services enable container.googleapis.com
!gcloud services enable containerregistry.googleapis.com
!gcloud services enable containerfilesystem.googleapis.com

## Register model on Vertex AI

Once everything is set up, you can already initialize the Vertex AI session via the `google-cloud-aiplatform` Python SDK as follows:

In [5]:
import os
from google.cloud import aiplatform

aiplatform.init(
    project=os.getenv("PROJECT_ID"),
    location=os.getenv("LOCATION"),
)

Since Gemma models are gated, you need to login into your Hugging Face Hub account with a read-access token either fine-grained with access to the gated model, or just overall read-access to your account. More information on how to generate a read-only access token for the Hugging Face Hub in the instructions at <https://huggingface.co/docs/hub/en/security-tokens>.

In [6]:
!pip install --upgrade --quiet huggingface_hub


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [7]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Then you can already "upload" the model i.e. register the model on Vertex AI. It is not an upload per se, since the model will be automatically downloaded from the Hugging Face Hub in the Hugging Face DLC for TGI on startup via the `MODEL_ID` environment variable, so what is uploaded is only the configuration, not the model weights.

Before going into the code, let's quickly review the arguments provided to the `upload` method:

* **`display_name`** is the name that will be shown in the Vertex AI Model Registry.

* **`serving_container_image_uri`** is the location of the Hugging Face DLC for TGI that will be used for serving the model.

* **`serving_container_environment_variables`** are the environment variables that will be used during the container runtime, so these are aligned with the environment variables defined by `text-generation-inference`, which are analog to the [`text-generation-launcher` arguments](https://huggingface.co/docs/text-generation-inference/en/basic_tutorials/launcher). Additionally, the Hugging Face DLCs for TGI also capture the `AIP_` environment variables from Vertex AI as in [Vertex AI Documentation - Custom container requirements for prediction](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements).

    * `MODEL_ID` is the identifier of the model in the Hugging Face Hub. To explore all the supported models you can check <https://huggingface.co/models?sort=trending&other=text-generation-inference>.
    * `NUM_SHARD` is the number of shards to use if you don't want to use all GPUs on a given machine e.g. if you have two GPUs but you just want to use one for TGI then `NUM_SHARD=1`, otherwise it matches the `CUDA_VISIBLE_DEVICES`.
    * `MAX_INPUT_TOKENS` is the maximum allowed input length (expressed in number of tokens), the larger it is, the larger the prompt can be, but also more memory will be consumed.
    * `MAX_TOTAL_TOKENS` is the most important value to set as it defines the "memory budget" of running clients requests, the larger this value, the larger amount each request will be in your RAM and the less effective batching can be.
    * `MAX_BATCH_PREFILL_TOKENS` limits the number of tokens for the prefill operation, as it takes the most memory and is compute bound, it is interesting to limit the number of requests that can be sent.
    * `HUGGING_FACE_HUB_TOKEN` is the Hugging Face Hub token, required as [`google/gemma-7b-it`](https://huggingface.co/google/gemma-7b-it) is a gated model.

* (optional) **`serving_container_ports`** is the port where the Vertex AI endpoint will be exposed, by default 8080.

For more information on the supported `aiplatform.Model.upload` arguments, check its Python reference at https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_upload.

In [8]:
print(os.getenv("CONTAINER_URI"))
#huggingface-text-generation-inference-cu121.2-2.ubuntu2204.py310

us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu124.2-3.ubuntu2204.py311


In [9]:
!gcloud auth configure-docker us-docker.pkg.dev


{
  "credHelpers": {
    "us-docker.pkg.dev": "gcloud",
    "gcr.io": "gcloud"
  }
}
Adding credentials for: us-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [10]:
from huggingface_hub import get_token

HF_MODEL = "google/paligemma-3b-mix-224" # change this
DISPLAY_NAME = "paligemma"
#HF_MODEL = "Salesforce/blip2-opt-2.7b"
#HF_MODEL = "liuhaotian/llava-v1.5-13b"
#HF_MODEL = "llava-hf/llava-v1.6-mistral-7b-hf"
#DISPLAY_NAME = "llava-1.6"


model = aiplatform.Model.upload(
    display_name=DISPLAY_NAME,
    serving_container_image_uri="us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-2.ubuntu2204.py310",
    serving_container_environment_variables={
        "MODEL_ID": HF_MODEL,
        "HUGGING_FACE_HUB_TOKEN": get_token(),
        "HF_HUB_ENABLE_HF_TRANSFER": "1",
        "NUM_SHARD": "1",
    },
    serving_container_ports=[8080],
)
model.wait()

Creating Model
Create Model backing LRO: projects/841337720906/locations/us-central1/models/770332140071026688/operations/5437461753883525120


KeyboardInterrupt: 

model = aiplatform.Model.upload(
    display_name="Llama-Vision-11B",
    serving_container_image_uri=os.getenv("CONTAINER_URI"),
    serving_container_environment_variables={
        "MODEL_ID": "meta-llama/Llama-3.2-11B-Vision-Instruct",
        "NUM_SHARD": "2",
        "MAX_INPUT_TOKENS": "512",
        "MAX_TOTAL_TOKENS": "1024",
        "MAX_BATCH_PREFILL_TOKENS": "1512",
        "HF_HUB_ENABLE_HF_TRANSFER": "1",
        "HUGGING_FACE_HUB_TOKEN": get_token(),
        "MESSAGES_API_ENABLED": "true",
    },
    serving_container_ports=[8080],
)

![Model on Vertex AI Model Registry](./assets/vertex-ai-model.png)

## Deploy model on Vertex AI

After the model is registered on Vertex AI, you need to define the endpoint that you want to deploy the model to, and then link the model deployment to that endpoint resource.

To do so, you need to call the method `aiplatform.Endpoint.create` to create a new Vertex AI endpoint resource (which is not linked to a model or anything usable yet).

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name=f"{DISPLAY_NAME}-endpoint")

![Vertex AI Endpoint created](./assets/vertex-ai-endpoint.png)

Now you can deploy the registered model in an endpoint on Vertex AI.

The `deploy` method will link the previously created endpoint resource with the model that contains the configuration of the serving container, and then, it will deploy the model on Vertex AI in the specified instance.

Before going into the code, let's quickly review the arguments provided to the `deploy` method:

- **`endpoint`** is the endpoint to deploy the model to, which is optional, and by default will be set to the model display name with the `_endpoint` suffix.
- **`machine_type`**, **`accelerator_type`** and **`accelerator_count`** are arguments that define which instance to use, and additionally, the accelerator to use and the number of accelerators, respectively. The `machine_type` and the `accelerator_type` are tied together, so you will need to select an instance that supports the accelerator that you are using and vice-versa. More information about the different instances at [Compute Engine Documentation - GPU machine types](https://cloud.google.com/compute/docs/gpus), and about the `accelerator_type` naming at [Vertex AI Documentation - MachineSpec](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec).

For more information on the supported `aiplatform.Model.deploy` arguments, you can check its Python reference at https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_deploy.

In [ ]:
deployed_model = model.deploy(
    endpoint=endpoint,
    machine_type="g2-standard-24",
    accelerator_type="NVIDIA_L4",
    accelerator_count=2,
)

In [ ]:
deployed_model.to_dict()

In [ ]:
model.predict_schemata

**WARNING**: _The Vertex AI endpoint deployment via the `deploy` method may take from 15 to 25 minutes._

## Online predictions on Vertex AI

Finally, you can run the online predictions on Vertex AI using the `predict` method, which will send the requests to the running endpoint in the `/predict` route specified within the container following Vertex AI I/O payload formatting.

As you are serving a `text-generation` model, you will need to make sure that the chat template, if any, is applied correctly to the input conversation; meaning that `transformers` need to be installed so as to instantiate the `tokenizer` for [`google/gemma-7b-it`](https://huggingface.co/google/gemma-7b-it) and run the `apply_chat_template` method over the input conversation before sending the input within the payload to the Vertex AI endpoint.

**Note:** The chat template might not be needed for Gemma-2b/the model PaliGemma is based on.

In [ ]:
!pip install --upgrade --quiet transformers

After the installation is complete, the following snippet will apply the chat template to the conversation:

>**This isn't needed for PaliGemma but might be needed for Gemma-2b/the LM it is based on.**


### Via Python

#### Within the same session

If you are willing to run the online prediction within the current session, you can send requests programmatically via the `aiplatform.Endpoint` (returned by the `aiplatform.Model.deploy` method) as in the following snippet:


Here, we have listed code to supply images to the model via two different ways: (1) via urls, and (2) from local files. KM has extrapolated this syntax from the TGI docs on huggingface: https://huggingface.co/docs/text-generation-inference/en/basic_tutorials/visual_language_models

In [ ]:
import base64
import requests
import io

**URL Based**

In [ ]:
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/rabbit.png"
url = "https://people.cs.umu.se/dali/test.png"

output = endpoint.predict(
    instances=[
        {
            "inputs":f"![]({url}) What is in this image?\n",
            "parameters":{"max_new_tokens": 100, "do_sample": False}
        }
    ]
)
# > helmet

In [ ]:
output

**Locally**

In [ ]:
image_path = "fgqa_hs/images/output.png"
image_path = "/home/dali/Downloads/rabbit.png"
from PIL import Image

# Open an image file
with open(image_path, "rb") as f:

    
    image = base64.b64encode(f.read()).decode("utf-8")

    #image = f"data:image/png;base64,{image}"

    output = deployed_model.predict(
        instances=[
            {"prompt": "Describe this image in detail:\n","base64_image":image}
        ]
    )
#> space suit

In [ ]:
# is different when image is passed from a local file vs. from an url
# even when we use greedy decoding (I think?), strange!
output

In [11]:
MODEL_ID = "liuhaotian/llava-v1.5-13b"
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-llava-serve"

def deploy_model(model_id: str) -> tuple[aiplatform.Model, aiplatform.Endpoint]:
   """Uploads and deploys the model to Vertex AI endpoint for prediction."""
   model_name = "llava_1.5"
   endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
   serving_env = {
       "MODEL_ID": model_id,
       "TS_NUM_WORKERS": 1,
       "PRECISION_MODE": "4bit",
       "DEPLOY_SOURCE": "custom",
   }
   # If the model_id is a GCS path, use artifact_uri to pass it to serving docker.
   artifact_uri = model_id if model_id.startswith("gs://") else None
   model = aiplatform.Model.upload(
       display_name=model_name,
       serving_container_image_uri=SERVE_DOCKER_URI,
       serving_container_ports=[7080],
       serving_container_predict_route="/predictions/llava_serving",
       serving_container_health_route="/ping",
       serving_container_environment_variables=serving_env,
       artifact_uri=artifact_uri,
   )
   deployed_model = model.deploy(
       endpoint=endpoint,
       machine_type="g2-standard-24",
       accelerator_type="NVIDIA_L4",
       accelerator_count=2,
       deploy_request_timeout=1800,
   )
   return model, deployed_model, endpoint

model, deployed_model, endpoint = deploy_model(model_id=MODEL_ID)

Creating Endpoint
Create Endpoint backing LRO: projects/841337720906/locations/us-central1/endpoints/8139500961983889408/operations/8933943914583293952
Endpoint created. Resource name: projects/841337720906/locations/us-central1/endpoints/8139500961983889408
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/841337720906/locations/us-central1/endpoints/8139500961983889408')
Creating Model
Create Model backing LRO: projects/841337720906/locations/us-central1/models/4762773209734971392/operations/2599912513663401984
Model created. Resource name: projects/841337720906/locations/us-central1/models/4762773209734971392@1
To use this Model in another session:
model = aiplatform.Model('projects/841337720906/locations/us-central1/models/4762773209734971392@1')
Deploying model to Endpoint : projects/841337720906/locations/us-central1/endpoints/8139500961983889408
Deploy Endpoint model backing LRO: projects/841337720906/locations/us-central1/endpoints/81395009619838

In [12]:
import base64
import io
import requests


from PIL import Image


def get_remote_image(path):
   """Downloads a remote image into memory."""
   response = requests.get(path)
   image = Image.open(io.BytesIO(response.content))
   return image


def image_to_base64(image: Image.Image) -> str:
   """Converts a PIL image to a base64 string."""
   buffer = io.BytesIO()
   image.save(buffer, format="JPEG")
   image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
   return image_str


prompt = 'What is in this image?'
image_path = 'https://people.cs.umu.se/dali/test.png'


image = get_remote_image(image_path)
instances = [
     {
         'prompt': prompt,
         'base64_image': image_to_base64(image)
     }
]
preds = endpoint.predict(instances=instances).predictions
print(preds)

[{'prompt': 'What is in this image?', 'base64_image': '...'}]
['The image features a rabbit wearing a space suit, standing on a rocky surface.']


Producing the following `output`:

```
Prediction(predictions=['space suit'], deployed_model_id='6484700777808920576', metadata=None, model_version_id='1', model_resource_name='projects/20178026/locations/us-central1/models/1030014796319162368', explanations=None)
```

In [15]:
def predict_answers(model, images, questions, gen_config):
    assert len(images) == len(questions)

    instances = []

    #generation_config = {
    #    "max_new_tokens": 256,
    #    "do_sample": True,
    #    "top_p": 0.2,
    #    "temperature": 0.2,
    #}

    binarized_images = []
    for i, image in enumerate(images):
        img = f"data:image/png;base64,{image}"
        instance = {"inputs":f"![]({img}){questions[i]}\n\n",
                "parameters": gen_config}
        instances.append(instance)
    print(len(instances[0]["inputs"]))
    output = model.predict(
        instances=instances
    )
    return output

In [16]:
image_path = "/home/dali/Downloads/rabbit.png"

generation_config = {"max_new_tokens": 100, "do_sample": False}

with open(image_path, "rb") as f:
    image = base64.b64encode(f.read()).decode("utf-8")
    images = [image, image]
    output = predict_answers(deployed_model, 
                             images, 
                             ["What is in the image?", "Is the animal in the image threatening?"], 
                             generation_config)
for prediction in output.predictions:
    print(prediction)
print(output)
#> space suit

561582


AttributeError: Unknown field for DeployedModel: predict

## Evaluation task

Based on [https://huggingface.co/docs/google-cloud/main/examples/vertex-ai-notebooks-evaluate-llms-with-vertex-ai](https://huggingface.co/docs/google-cloud/main/examples/vertex-ai-notebooks-evaluate-llms-with-vertex-ai)

In [81]:
from datasets import load_dataset

dataset = load_dataset("fgqa_hs", split='test[:100]')

dataset

Dataset({
    features: ['question', 'answer', 'argument', 'image', 'substitution'],
    num_rows: 100
})

In [68]:
dataset[0]

{'question': 'Is the curtain on the right side or on the left of the picture?',
 'answer': 'right',
 'argument': 'curtain',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x334>,
 'substitution': ''}

In [69]:
#dataset = dataset.select(range(100))

In [70]:
#dataset

We must convert to a pandas dataset in order to use the Vertex Evaluation API

In [82]:
df = dataset.to_pandas()

In [72]:
def binarize_image(img_path, img_type='jpg'):
    
    with open(f"{img_path}", "rb") as f:
        image = base64.b64encode(f.read()).decode("utf-8")
    return f"data:image/{img_type};base64,{image}"

Step 0. Convert images to base64 representations

In [83]:
from PIL import Image
df["base64_image"] = df["image"].apply(lambda x: image_to_base64(Image.open(x['path'])))

In [84]:
#df['prompt'] = df.apply(lambda row: f"![]({row['img']}) {row['question']}", axis=1)
df['prompt'] = df.apply(lambda row: (row['base64_image'], f"Answer with a single word or concept. {row['question']}"), axis=1)

df['prompt'][1][1]

'Answer with a single word or concept. Is the blind on the right side or on the left of the picture?'

In [85]:
df['reference'] = df['answer']

Drop all columns that we do not need for the prediction task

In [76]:
def generate(prompt, generation_config=generation_config):

    payload = prompt_to_payload(prompt, generation_config)
    output = endpoint.predict(instances=[payload])
    generated_text = output.predictions[0]
    #print(output.predictions)
    return generated_text.lower()

def prompt_to_payload(prompt, generation_config):
    return {"base64_image": prompt[0],"prompt": prompt[1]}


In [77]:
prompt_data = []
for row in df['prompt']:
    prompt_data.append(prompt_to_payload(row,{}))

In [78]:
import json
with open("data.json", 'w') as f:
    for item in prompt_data:
        f.write(json.dumps(item) + "\n")

In [80]:
generate(df['prompt'][0])

'right'

In [86]:
from vertexai.evaluation import EvalTask
from vertexai.generative_models import (Part)
# 2. create eval task
eval_task = EvalTask(
        dataset=df,
        metrics=["exact_match"],
        experiment="multimodal-hypernym-semantics",
)

In [ ]:
import uuid
# 3. run eval task
# Note: If the last iteration takes > 1 minute you might need to retry the evaluation
exp_results = eval_task.evaluate(
        model=generate, experiment_run_name=f"test-gqa-{str(uuid.uuid4())[:8]}"
)

Associating projects/841337720906/locations/us-central1/metadataStores/default/contexts/multimodal-hypernym-semantics-test-gqa-4a17fc36 to Experiment: multimodal-hypernym-semantics


Generating a total of 100 responses from the custom model function.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:52<00:00,  1.73s/it]

All 100 responses are successfully generated from the custom model function.
Multithreaded Batch Inference took: 172.8143997840234 seconds.
Computing metrics with a total of 100 Vertex Gen AI Evaluation Service API requests.



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:41<00:00,  4.02s/it]

All 100 metric requests are successfully computed.
Evaluation Took:401.9961060429923 seconds


In [ ]:
df

In [ ]:

def predict(prompts, generation_config=generation_config):
    payloads = [prompt_to_payload(prompt, generation_config) for prompt in prompts]
    print(payloads)
    output = endpoint.predict(instances=payloads)
    generated_texts = output.predictions
    #print(output.predictions)
    return [pred.lower() for pred in generated_texts]

In [ ]:

from tqdm import tqdm
predictions = []
for index, row in tqdm(df.iterrows(), total=len(df['question'])):
    pred = generate([row['base64_image'], row['question']])        
    predictions.append(pred)
df['online_responses'] = predictions

In [ ]:
df['online_responses']

In [ ]:
results = {}
print(exp_results.summary_metrics)
print(f"{exp_results.summary_metrics['exact_match/mean']}")
results["test"] = exp_results.summary_metrics["exact_match/mean"]

for prompt_name, score in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{prompt_name}: {score}")

In [ ]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def get_url_from_gcs(gcs_uri: str) -> str:
    # converts GCS uri to url for image display.
    url = "https://storage.googleapis.com/" + gcs_uri.replace("gs://", "").replace(
        " ", "%20"
    )
    return url


def print_multimodal_prompt(contents: list):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if isinstance(content, Image):
            display_images([content])
        elif isinstance(content, Part):
            url = get_url_from_gcs(content.file_data.file_uri)
            IPython.display.display(load_image_from_url(url))
        else:
            print(content)

### Check predictions

In [ ]:
exp_results.metrics_table[['question', 'response', 'reference', 'argument', 'substitution', 'exact_match/score']]
#exp_results.metrics_table[['question', 'response', 'reference', 'argument', 'substitution', 'exact_match/score', 'rouge/score']]

In [ ]:
result_df = exp_results.metrics_table[['question', 'response', 'reference', 'argument', 'substitution', 'exact_match/score']]
#result_df = exp_results.metrics_table[['question', 'response', 'reference', 'argument', 'substitution', 'exact_match/score', 'rouge/score']]

In [ ]:
result_df.to_csv('llava-100-test.csv')

In [ ]:
aggregated_base_questions = result_df[result_df['substitution'] == ''].groupby('argument').agg({
    'exact_match/score': 'mean',
  #  'rouge/score': 'mean'
}).reset_index()

In [ ]:
aggregated_base_questions

# Aggregating over all substitutions

In [ ]:
aggregated_substitutions = result_df[result_df['substitution'] != ''].groupby('argument').agg({
    'exact_match/score': 'mean',
#    'rouge/score': 'mean'
}).reset_index()

In [ ]:
aggregated_substitutions

In [ ]:
import pandas as pd
aggregated_combined = aggregated_base_questions.rename(columns={'exact_match/score': 'base/exact_match/score',
                                                                #'rouge/score':'base/rouge/score'
                                                               })

# Merge the two dataframes on a common key (in this case, 'key')
aggregated_combined = pd.merge(aggregated_combined, aggregated_substitutions, on='argument', how='left')

# Fill empty values with 0.0
aggregated_combined = aggregated_combined.fillna(0.0)

In [ ]:
aggregated_combined

In [ ]:
print(exp_results.metrics_table['response'])

In [ ]:
from IPython.display import Image

from PIL import Image as PImage

imgs = set([img['path'] for img in df['image'][:10]])

for image in imgs:
    #Image(filename=image['path'])
    img = mpimg.imread(image)
    plt.imshow(img)
    plt.show()
    img = image_to_base64(PImage.open(image))
    output = generate([img, "What is in the image?"])
    print(output)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
image = mpimg.imread(image['path'])
plt.imshow(image)
plt.show()


# Batch inference

Below is example code from the GCP documentation found at (https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions)[https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions]

Also check (https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/batch_eval_llm.ipynb)[https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/batch_eval_llm.ipynb]
    

In [ ]:
def create_batch_prediction_job_dedicated_resources_sample(
    model,
    job_display_name: str,
    gcs_source,
    gcs_destination: str,
    machine_type="g2-standard-24", #$0.8129 USD / hour
    accelerator_type="NVIDIA_L4", #$0.644046 USD / hour
    accelerator_count=2,
    instances_format: str = "jsonl",
    starting_replica_count: int = 1,
    max_replica_count: int = 1,
    sync: bool = True,
):


    batch_prediction_job = model.batch_predict(
        job_display_name=job_display_name,
        gcs_source=gcs_source,
        gcs_destination_prefix=gcs_destination,
        instances_format=instances_format,
        starting_replica_count=starting_replica_count,
        max_replica_count=max_replica_count,
        
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        sync=sync,
    )

    batch_prediction_job.wait()

    print(batch_prediction_job.display_name)
    print(batch_prediction_job.resource_name)
    print(batch_prediction_job.state)
    return batch_prediction_job

In [ ]:
batch_prediction_job = create_batch_prediction_job_dedicated_resources_sample(
    model,
        job_display_name="batch-llava-test-100",
        gcs_source="gs://multimodal-representations-eval-data/data.jsonl",
        gcs_destination="gs://multimodal-representations-eval-data/",
)


## Resource clean-up (DEFINITELY DO THIS)

Finally, you can already release the resources that you've created as follows, to avoid unnecessary costs:

* `deployed_model.undeploy_all` to undeploy the model from all the endpoints.
* `deployed_model.delete` to delete the endpoint/s where the model was deployed gracefully, after the `undeploy_all` method.
* `model.delete` to delete the model from the registry.

In [ ]:
deployed_model.undeploy_all()
deployed_model.delete()
model.delete()

Alternatively, you can also remove those from the Google Cloud Console following the steps:

* Go to Vertex AI in Google Cloud
* Go to Deploy and use -> Online prediction
* Click on the endpoint and then on the deployed model/s to "Undeploy model from endpoint"
* Then go back to the endpoint list and remove the endpoint
* Finally, go to Deploy and use -> Model Registry, and remove the model

In [ ]:
# Disable APIs

!gcloud services disable aiplatform.googleapis.com
!gcloud services disable compute.googleapis.com
!gcloud services disable container.googleapis.com
!gcloud services disable containerregistry.googleapis.com
!gcloud services disable containerfilesystem.googleapis.com

### PLEASE ALSO MANUALLY ENSURE ALL APIS ARE DISABLED ON GCP AFTER THIS IS DONE!


In [ ]:
# Download an image from Google Cloud Storage
# Load from local file
from vertexai.generative_models import Image as V_Image

gen_model = GenerativeModel("paligemma")

image = V_Image.load_from_file(df['image'][0]['path'])

# Prepare contents
prompt = "Describe this image?"
contents = [image, prompt]

response = gen_model.generate_content(contents)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Response--------")
print(response.text)

In [ ]:
endpoints = aiplatform.Endpoint.list()
for i in endpoints:
        i.undeploy_all()

# Alternatives for PaliGemma

https://ai.google.dev/gemma/docs/paligemma/inference-with-keras